# Report

## Introduction and data

> REMOVE THE FOLLOWING TEXT

This section includes an introduction to the project motivation, data, and (research) question.

> Use content from the [BIG IDEA worksheet](https://docs.google.com/document/d/1-GZvhdbhLYLB_Bo1arj1rgTqbJ5SUoU21vtgbYEhVqk/edit?usp=sharing)   

Describe the data and definitions of key variables.

It should also include some exploratory data analysis.

*All of the EDA won't fit in the paper, so focus on the EDA for the response variable and a few other interesting variables and relationships.*


////
### Subject Matter:

In the city of Chicago, many incidents/crimes happen every day, from minor thefts to murders. To reduce the violence in the city, the city wants to open a new crime prevention centre. Now the city is asking our team which crimes occur particularly frequently and where they happen. With this information, the **Crime Prevention Center** can be built in a particularly well-situated location. In addition, the specialised departments of the centre can be trained for the relevant criminal offences. This should make Chicago a safer city and ensure that measures are taken at an early stage to prevent crime.

### Motivation:

Various studies show that it is possible to prevent crime in cities with the help of specific actions. With the new **Crime Prevention Center**, we want to take a new approach in Chicago to prevent crime from the very beginning.

* Crime Prevention and the Safer Cities Story
https://onlinelibrary.wiley.com/doi/10.1111/j.1468-2311.1993.tb00758.x

* Social Crime Prevention in South Africa's Major Cities 
http://csvr.org.za/docs/urbansafety/socialcrimeprevention.pdf




### General Question:

Which kind of crimes happen particularly frequently and where do they happen?

### Hypotheses:

There are places (districts/blocks) in Chicago where the most (dangerous) crimes/incidents happen.


In [ ]:
from pathlib import Path

PARENT_PATH = str(Path().resolve().parent) + "/"
PATH = "data/"
SUBPATH = "processed/"
FILE = "chicago_crimes-20230130-1108"
FORMAT = ".csv"

In [ ]:
import altair as alt
from vega_datasets import data
alt.data_transformers.disable_max_rows()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import pandas as pd

df = pd.read_csv(PARENT_PATH + PATH + SUBPATH + FILE + FORMAT)

In [ ]:
df.head()

In [ ]:
df.info()

### Description of the crimes
There were 76362 registered crimes in 2018 and 2019. "theft" was the most common with 21637 cases committed.

In [ ]:
df["primary_type"].describe()

### Distribution of crimes over the two years (2018 + 2019)
The distribution of crimes are almost equal, there were slightly more crimes in 2018. 

In [ ]:
chart_3 = alt.Chart(df).mark_bar().encode(
    y=alt.Y('year:N',
            axis=alt.Axis(title="YEAR",
                          titleAnchor="start")),
    x=alt.X('count(primary_type)',
            axis=alt.Axis(title = "COUNT", 
                          titleAnchor="start")),
).properties(
    title='Count of commited crime per year',
    width=400,
    height=200
)


chart_3.configure_title(
    fontSize=16,
    font='Arial',
    color='black',
    anchor='start'
).configure_axis(grid=False
).configure_view(strokeOpacity=0)

### Distribution of crimes per year in months
Most crimes occour in spring/summer from May to August.

In [ ]:
chart_4 = alt.Chart(df).mark_line().encode(
    x=alt.X('month:N',
            axis=alt.Axis(title="MONTH",
                          titleAnchor="start", 
                          labelAngle=0)),
    y=alt.Y('count(primary_type)',
            axis=alt.Axis(title = "COUNT", 
                          titleAnchor="end")),
    color=alt.Color("year:N", legend=alt.Legend(title="YEAR"))                  
).properties(
    title='Count of commited crime per month',
    width=600,
    height=400
).configure_axis(grid=False
).configure_view(strokeOpacity=0)

chart_4.configure_title(
    fontSize=16,
    font='Arial',
    color='black',
    anchor='start'
)

### Distribution of crimes per day in hours
- Most crimes occour during the day.
- Constant increase from 8 am to 7 pm.
- Peak at 12pm
- Constant decrease from 7pm to 1 am.
- Lowest at 5 am

In [ ]:
chart_5 = alt.Chart(df).mark_line().encode(
    x=alt.X('hour:N',
            axis=alt.Axis(title="HOUR",
                          titleAnchor="start",
                          labelAngle=0)),
    y=alt.Y('count(primary_type)',
            axis=alt.Axis(title = "COUNTHO", 
                          titleAnchor="end")),
    color=alt.Color("year:N", legend=alt.Legend(title="YEAR"))
).properties(
    title='Count of commited crime per hour',
    width=600,
    height=400
).configure_axis(grid=False
).configure_view(strokeOpacity=0)


chart_5.configure_title(
    fontSize=16,
    font='Arial',
    color='black',
    anchor='start'
)

### Distribution of the crimes by type

- most: "theft" and "assault_and_battery"
- least: "sexual_crime" and "homicide"

In [ ]:
df.primary_type.value_counts()

In [ ]:
ch = alt.Chart(df).mark_bar().encode(
    x=alt.X("primary_type", sort="-y",
            axis=alt.Axis(title="DISTRICT",
                          titleAnchor="start", 
                          labelAngle=0)),
    y=alt.Y('count(primary_type)', 
            axis=alt.Axis(title = "TYPE", 
                        titleAnchor="end"),
                        scale=alt.Scale(domain=[0, 24000])),
).properties(
    title='Count of commited crimes per type',
    width=1000,
    height=400
)

txt = ch.mark_text(
    baseline = 'middle',
    dy= - 15
).encode(
    text='count(primary_type)'
)

layer = alt.layer(ch + txt
).configure_title(
    fontSize=16,
    font='Arial',
    color='black',
    anchor='start'
).configure_axis(grid=False
).configure_view(strokeOpacity=0)

layer

### Distribution of crimes per block/street

In [ ]:
df["block"].value_counts().nlargest(10)

### Distribution of crime type per disctrict in percent

In [ ]:
#primary_type and district crosstab

cross_table = pd.crosstab(df["primary_type"], df["district"],
    margins=True,
    normalize=True,
    rownames=["Type"],
    colnames=["District"]
    )* 100


cross_table

In [ ]:
df["district"].value_counts(normalize=True) * 100

### Distribution of crime type per disctrict in total

Top 5 Districts:

In [ ]:
df["district"].value_counts().nlargest(5)

In [ ]:
district = alt.Chart(df).mark_bar().encode(
    x=alt.X("district:N",
    sort="-y",
    axis=alt.Axis(title="DISTRICT",  
                          titleAnchor="start", 
                          labelAngle=0, grid=False)),
    y=alt.Y("count(primary_type)",
    axis=alt.Axis(title = "COUNT", 
                        titleAnchor="end")),
    tooltip=['district', alt.Tooltip('count(primary_type)', title='count')]
).properties(
    title='Count of commited crimes per district',
    width=1000,
    height=400
).configure_axis(grid=False
).configure_view(strokeOpacity=0)



district.configure_title(
    fontSize=16,
    font='Arial',
    color='black',
    anchor='start'
).configure_axis(grid=False
).configure_view(strokeOpacity=0)

### Distribution of crime groups in total

Since 10 single crime types were too many for some purposes (map, etc.), we decided to combine the types into groups.

- Group_1: light to medium crimes
- Group_2: medium to serious crimes
- Group_3: homicide/murder


In [ ]:
#Barchart with different groups of crime
# we bulit these groups sorted according to gravity

ch = alt.Chart(df).mark_bar().encode(
    y=alt.Y('count(primary_group)', 
            axis=alt.Axis(title = "COUNT", 
                        titleAnchor="end")),
    x=alt.X("primary_group", sort="-y",
            axis=alt.Axis(title="GROUP",
                            labelAngle=0,
                          titleAnchor="start")),

).properties(
    title='Count of commited crimes per group',
    width=800,
    height=400
)

txt = ch.mark_text(
    baseline = 'middle',
    dy = -15
).encode(
    text='count(primary_type)'
)

layer = alt.layer(ch + txt
).configure_title(
    fontSize=16,
    font='Arial',
    color='black',
    anchor='start'
).configure_axis(grid=False
).configure_view(strokeOpacity=0)

layer


### Arrest

In [ ]:
#arrest crime crosstab in total

cross_table = pd.crosstab(df["primary_type"], df["arrest"],
    margins=True,
    normalize=False,
    rownames=["Crime"],
    colnames=["Arrest"]
    )


cross_table

In [ ]:
#arrest crime crosstab in percent

cross_table = pd.crosstab(df["primary_type"], df["arrest"],
    margins=True,
    normalize=True,
    rownames=["Crime"],
    colnames=["Arrest"]
    )* 100


cross_table

In [ ]:
#arrest homocide crosstab

cross_table = pd.crosstab(df["primary_type"] =="homicide" , df["arrest"],
    margins=True,
    normalize=True,
    rownames=["Crime"],
    colnames=["Arrest"]
    )* 100


cross_table

In [ ]:
#arrest group crosstab

cross_table = pd.crosstab(df["primary_group"], df["arrest"],
    margins=True,
    normalize=True,
    rownames=["Group"],
    colnames=["Arrest"]
    )* 100


cross_table

In [ ]:
homicide = alt.Chart(df).mark_bar().encode(
    x=alt.X("count(primary_type)"),
    y=alt.Y("arrest")
).transform_filter(
alt.FieldEqualPredicate(field='primary_type', equal="homicide")
).configure_axis(grid=False
).configure_view(strokeOpacity=0)



homicide

## Visualizations

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your visualization creation process.

Explain the reasoning for the type of visualization you're using and what other types you considered. 

Additionally, show how you arrived at the final visualization by describing the plot selection process, variable transformations (if needed), and any other relevant considerations that were part of the visualization creation process.

In [ ]:
#Geopandas library to work with Chicago map
import geopandas as gpd

In [ ]:
PARENT_PATH = str(Path().resolve().parent) + "/"
PATH = "data/"
SUBPATH = "external/"
FILE = "wards"
FORMAT = ".shp"

gdf = gpd.read_file(PARENT_PATH + PATH + SUBPATH + FILE + FORMAT)

In [ ]:
#Map of Chicago with Crimes as Dots on the Map

choro = alt.Chart(gdf).mark_geoshape(
    fill="black", stroke='white'
).encode()


p = alt.Chart(df).mark_square(opacity=0.3).encode(
    longitude='longitude',
    latitude='latitude',
    size=alt.value(10),
    color="count(primary_type)",
    tooltip=['ward', "district", "block"]
).properties(
    title="Location of crimes in Chicago City",
    width=1000,
    height=1000
)

choro + p

In [ ]:

# select a point for which to provide details-on-demand
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

chart_5 = alt.Chart().mark_line().encode(
    x=alt.X('hour:N',
            axis=alt.Axis(title="Hour",
                          titleAnchor="start",
                          labelAngle=0)),
    y=alt.Y('count(primary_type)',
            axis=alt.Axis(title = "Count", 
                          titleAnchor="end")),
    color=alt.Color("year:N", legend=alt.Legend(title="YEAR"))
)


alt.layer(
    chart_5,
    alt.Chart().mark_rule(color='lightgrey').encode(
        x='hour:N'
    ).transform_filter(label),

chart_5.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

chart_5.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='count(primary_type)'
    ).transform_filter(label),

chart_5.mark_text(align='left', dx=5, dy=-5).encode(
        text='count(primary_type)'
    ).transform_filter(label),
    data=df
).properties(
    width=800,
    height=600
).configure_axis(grid=False
).configure_view(strokeOpacity=0)

In [ ]:
district_5 = alt.Chart(df).mark_bar().encode(
    x=alt.X("district:N",
    sort="-y",
    axis=alt.Axis(title="DISTRICT",  
                          titleAnchor="start", 
                          labelAngle=0)),
    y=alt.Y("count(primary_type):Q"),
    color=alt.condition(
        alt.FieldOneOfPredicate('district', [11, 6, 8, 1, 18]),  # If the district is 11 this test returns True,
        alt.value('orange'),     # which sets the bar orange.
        alt.value('steelblue')   # And if it's not true it sets the bar steelblue.
    )
).properties(
    title='Count of commited crime in the districts',
    width=1000,
    height=400
).configure_axis(grid=False
).configure_view(strokeOpacity=0)



district_5

In [ ]:
#primary_type per district


alt.Chart(df).mark_bar().encode(
    x=alt.X('count(primary_type)', stack="normalize"),
    y='district:N',
    color='primary_type',
    tooltip=["primary_type"]
).properties(
    title='Distribution of primary_type per district',
    width=1000,
    height=800
).configure_axis(grid=False
).configure_view(strokeOpacity=0)

In [ ]:
# filter per district
districts = df['district'].unique() # get unique field values
districts = list(filter(lambda d: d is not None, districts)) # filter out None values
districts.sort() # sort alphabetically

selectDistrict = alt.selection_single(
    name='Select', # name the selection 'Select'
    fields=['district'], # limit selection to the Major_Genre field
    init={'district': districts[0]}, # use first genre entry as initial value
    bind=alt.binding_select(options=districts) # bind to a menu of unique genre values
)


chart = alt.Chart(df).mark_bar().add_selection(
    selectDistrict
).encode(
    x=alt.X("district:N",
    axis=alt.Axis(title="DISTRICT",  
                          titleAnchor="start", 
                          labelAngle=0, grid=False)),
    y=alt.Y("count(primary_type)"),
    opacity=alt.condition(selectDistrict, alt.value(1.0), alt.value(0.50)),
    tooltip=["district", "count(primary_type)"]
).properties(
    title='Count of commited crime in the districts',
    width=1000,
    height=400
).configure_axis(grid=False
).configure_view(strokeOpacity=0)



chart

## Conclusion + recommended action


> REMOVE THE FOLLOWING TEXT

In this section you'll include a summary of what you have learned about your (research) question along with (statistical) arguments supporting your conclusions.

In addition, discuss the limitations of your analysis and provide suggestions on ways the analysis could be improved.

Any potential issues pertaining to the reliability and validity of your data and appropriateness of the statistical analysis should also be discussed here.

Lastly, this section will include your recommended action.